In [ ]:
import malariagen_data
import os 

ag3 = malariagen_data.Ag3(pre=True)

In [ ]:
ag3.biallelic_snps_to_plink(output_dir=os.getcwd(),
                            region='2L:100000-2000000',
                            n_snps=2000,
                            sample_sets='AG1000G-AO',
                            )